# Crear un TorchPlayer


Recibe el modelo a instanciar como path y juega con el mismo

- Pensar como resolver el problema de que solo samplee las válidas
- Agregarle la opción de monte carlo tree search (opcional) con las opciones de iterationLimit, timeLimit

Si va a agregar MCTS mirar la notebook 007_MCTS.ipnb

In [3]:
class TorchPlayer():
    # 
    def __init__(self, model_path, player=1, board_shape=None, env=None, deterministic=True, only_valid=True, mcts=False, iterationLimit=None, timeLimit=None):
        pass
    
    def predict(self, board):
        pass

# Arena

Testear el jugador contra los distintos jugadore